In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import smelly_rats

# Loading the data

In [ ]:
xl = pd.ExcelFile('../data/onions.xls')

## HNMR spectra

In [ ]:
hnmr_spectra = (
    pd.read_excel(xl, sheet_name=0, index_col=0)
    .rename(columns=lambda col: col.replace("'", ""))
)
print('HNMR spectra shape:', hnmr_spectra.shape)
hnmr_spectra.head()

**Spectra range**

In [ ]:
print('Spectra is ranging from (ppm)')
hnmr_spectra.index.min(), hnmr_spectra.index.max()

As set in the paper the values of the spectra between 5 and 4.50 ppm are removed. Approximately, it is actually `5.027118` to `4.598711`.

In [ ]:
hnmr_spectra[5.03:5]

In [ ]:
hnmr_spectra[5:4.5]

**Columns**

In [ ]:
print(f'There are {len(hnmr_spectra.columns)} columns')
hnmr_spectra.columns

In [ ]:
hnmr_spectra.plot(y='H2-12-')

**The following columns are duplicates, but the values (of the column) are not duplicates.**

In [ ]:
double_columns = [col for col in hnmr_spectra.columns if col.startswith("H4-28")]
print(double_columns)
# [list(hnmr_spectra.columns).index(c) for c in double_columns]
hnmr_spectra[double_columns]

**Column groups**

In [ ]:
from pprint import pprint
groups = (1, 2, 3, 4)
group_cols = {
    group: [c for c in hnmr_spectra.columns if c.startswith(f'H{group}')]
    for group in groups
}
pprint(group_cols)

Reproducing fig 2 of paper

In [ ]:
fig, axes = plt.subplots(len(groups), 2, figsize=(10, 8))

for idx, group in enumerate(groups):
    ax = axes[idx, 0]
    hnmr_spectra[group_cols[group]].mean(axis=1).plot(ax=ax, label='')
    ax.set_ylim([0, 50_000])
    ax.set_xlim([9, 0])
    ax.set_title(f'Group {group}')
    
    ax = axes[idx, 1]
    hnmr_spectra[group_cols[group]].mean(axis=1).plot(ax=ax, label='')
    ax.set_title(f'Group {group}')
    ax.set_ylim([0, 7500])
    ax.set_xlim([9.5, 6])
    
fig.legend(loc="center right", borderaxespad=0.1)
fig.tight_layout()

## Loading the target (i.e. onion% groups)

In [ ]:
target = pd.read_excel(xl, sheet_name=1, usecols=range(10, 15))
print(target.shape)
target.head()

# Pareto scaling

In [ ]:
from smelly_rats import preproccessing

In [ ]:
hnmr_spectra_scaled = (
    hnmr_spectra
    .apply(preproccessing.pareto_scaling)
)

In [ ]:
fig, axes = plt.subplots(len(groups), 2, figsize=(10, 8))

for idx, group in enumerate(groups):
    ax = axes[idx, 0]
    hnmr_spectra_scaled[group_cols[group]].mean(axis=1).plot(ax=ax, label='')
    ax.set_ylim([0, 500])
    ax.set_xlim([9, 0])
    ax.set_title(f'Group {group}')
    
    ax = axes[idx, 1]
    hnmr_spectra_scaled[group_cols[group]].mean(axis=1).plot(ax=ax, label='')
    ax.set_title(f'Group {group}')
    ax.set_ylim([0, 500])
    ax.set_xlim([9.5, 6])
    
fig.legend(loc="center right", borderaxespad=0.1)
fig.tight_layout()

# Applying different dimentionality reduction techniques

In [ ]:
import numpy as np
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
from sklearn.decomposition import PCA

In [ ]:
for reducer in PCA, PLSCanonical, PLSRegression, CCA:
    red = reducer(n_components=2)
    transformed = red.fit_transform(hnmr_spectra_scaled.T, target['y'])
    try: 
        hnmr_spectra_reduced, _ = transformed
    except ValueError:
        hnmr_spectra_reduced = transformed

    fig, ax = plt.subplots(figsize=(10, 8))

    symbols = ['s', 'o', '*', '<']
    for idx, cat in enumerate(sorted(target['y'].unique())):
        mask = target['y'] == cat
        x_values = hnmr_spectra_reduced[mask, 0]
        y_values = hnmr_spectra_reduced[mask, 1]
        ax.plot(x_values, y_values, symbols[idx], label=cat, ms=10)

        ax.axhline(0, linestyle='--', color='gray', alpha=0.2)
        ax.axvline(0, linestyle='--', color='gray', alpha=0.2)

        x_max = np.abs(x_values).max()
        ax.set_xlim([-x_max, x_max])
        y_max = np.abs(y_values).max()
        ax.set_ylim([-y_max, y_max])
        
    ax.set_title(reducer.__name__)
    fig.legend(loc="center right", borderaxespad=0.1)
    fig.tight_layout()

# Classification on reduced data

Only using the 2 components here. (Will probably not be accurate judging from the figures above.)

In [ ]:
red = PCA(n_components=2)
hnmr_spectra_reduced = red.fit_transform(hnmr_spectra_scaled.T, target['y'])

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

reg = linear_model.Lasso(alpha=0.1)

scores = cross_val_score(
    reg, 
    hnmr_spectra_reduced,
    target['y'], 
    cv=5
)
scores

In [ ]:
from sklearn.linear_model import LogisticRegression



reg = LogisticRegression(
    penalty='l1', 
    solver='saga', 
    multi_class='multinomial', 
    C=1e-10, 
#     max_iter=1000
)

scores = cross_val_score(
    reg, 
    hnmr_spectra_reduced, 
    target['y'], 
    cv=5
)
scores

**Only convergence for small `C`**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-10, -6, num=5)
}

mod = GridSearchCV(
    reg,
    param_grid,
    cv=5,
    return_train_score=True,
    iid=False,
)

mod.fit(hnmr_spectra_reduced, target['y'])

# Interval PLS

## Custom scorers

In [ ]:
from sklearn.metrics import confusion_matrix, make_scorer

def misclassification(y, y_hat):
    y = y.values if hasattr(y, 'values') else y
    return (y.flatten() != y_hat.flatten().round().astype(int)).sum()


def rmse(y, y_hat):
    y = y.values if hasattr(y, 'values') else y
    return np.sqrt(np.square(y.flatten() - y_hat.flatten()).mean())


misclassification_scorer = make_scorer(misclassification)
rmse_scorer = make_scorer(rmse)

## Interval pls

In [ ]:
######################################################################
######### Settings
######################################################################

# Number of intervals
num_intervals = 20

# PLS type
reg = PLSRegression
# reg = PLSCanonical
n_components = 11
scale = True

# scoring method
scoring = rmse_scorer
# scoring = misclassification_scorer

y = target['y']

######################################################################
######### Code
######################################################################

fig, ax = plt.subplots(figsize=(10, 8))

intervals = np.linspace(
    hnmr_spectra_scaled.index.max(), 
    hnmr_spectra_scaled.index.min(), 
    num_intervals + 1
)

# Global Model
scores = cross_val_score(
    reg(n_components=n_components, scale=scale),
    hnmr_spectra_scaled.T,
    y,
    scoring=scoring,
    cv=5
)
global_score, yerr = np.mean(scores), np.std(scores)

ax.axhline(global_score, linestyle='--', color='black', alpha=0.8)
ax.axhline(global_score - yerr, linestyle='-.', color='black', alpha=0.8)
ax.axhline(global_score + yerr, linestyle='-.', color='black', alpha=0.8)

# Interval models
max_height = 0
for start, end in zip(intervals[:-1], intervals[1:]):
    
    scores = cross_val_score(
        reg(n_components=n_components, scale=scale),
        hnmr_spectra_scaled[start:end].T,
        y,
        scoring=scoring,
        cv=5
    )

    height = np.mean(scores)
    yerr = np.std(scores)
    alpha = 0.3 if height > global_score else 0.8
    ax.bar(
        end, 
        height=height, 
        width=(start - end), 
        align='edge', 
        color='grey',
        alpha=alpha,
        edgecolor='black',
        yerr=yerr,
        capsize=7,
    )
    max_height = max(max_height, height)
        
# Also plot spectra for clarity
plotr = (
    hnmr_spectra_scaled
    .mean(axis=1)
    .transform(lambda series: series / series.max())
    .transform(lambda series: series * max_height)
)
plotr.plot(ax=ax, color='black')

ax.set_xlim([intervals.max(), intervals.min()])
ax.set_xlabel('ppm')
ax.set_ylabel('RMSECV')
ax.set_title(f'Interval {reg.__name__} on the H NMR spectra with {num_intervals} groups and {n_components} latent variables')
ax.set_xticks(np.arange(np.ceil(intervals.min()), np.ceil(intervals.max())))
fig.tight_layout();